In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import psutil
import copy
import pickle
import re
from abc import ABC, abstractmethod
import math
import copy
from copy import deepcopy
from pathlib import Path
from skimage import data
from skimage.filters import threshold_otsu
from skimage.color import rgb2gray
import scipy
from scipy.special import softmax
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
import skimage
import csv
import PIL
from skimage.filters import threshold_otsu
import torchvision.models as torchmodels
import openslide
import torch.utils.data
import torchvision
list_pathstoadd = ["../../../../",
                   "../../../",\
                   "../../../Src/BoVWPipeline/",
                   "../../../../PyDmed/"]
for path in list_pathstoadd:
    if(path not in sys.path):
        sys.path.append(path)
import projutils.datasets
import pydmed
from pydmed.utils.data import *
import pydmed.lightdl
from pydmed.lightdl import *
import pydmed.stat
from pydmed.stat import *
import localsrc_ipmi2021_warwickher2
from localsrc_ipmi2021_warwickher2 import *


In [2]:
#settings ========

fname_metainf = "50000"
mode_trainortest = "test" #fixed
val_randomseed_evaluation = 10
random.seed(val_randomseed_evaluation)

In [3]:
#load information stored in the metainf file =====
file_metainf = open("../DA_warwick_private/Trained_Models/250wsi_p2w_globalstage/metainf_src_{}.pkl".format(fname_metainf), 'rb')


pkl_metainf = pickle.load(file_metainf)
file_metainf.close()
kwargs_getdataset = pkl_metainf['kwargs_getdataset']
kwargs_model = pkl_metainf['kwargs_model']
kwargs_optimizer = pkl_metainf['kwargs_optimizer']
traininginfo = pkl_metainf['traininginfo']


In [4]:
#settings ====================
idx_split = traininginfo["idx_split"]
num_visualwords = kwargs_model["num_visualwords"]
scale_thumbnail = kwargs_getdataset["scale_foreground"]
batchsize = traininginfo["batchsize"]

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
tstart_notebook = time.time()

cuda:0


In [6]:
#load datasets ======================
train_ds, val_ds, test_ds = projutils.datasets.get_warwickher2(
             idx_split=idx_split,
             scale_foreground=scale_thumbnail
            )
#load model ===========================
model = PipelineWarwickHER2(num_classes=kwargs_model["num_classes"],
                        num_visualwords=num_visualwords,\
                        device_stg3=device,\
                        size_input=kwargs_model["size_input"]) 


model_dict = model.state_dict()
checkpoint = torch.load("../DA_warwick_private/Trained_Models/250wsi_p2w_globalstage/model_src_{}.pt".format(fname_metainf))


# print(checkpoint.keys())
missing_keys = [k for k in model.state_dict() if k not in checkpoint.keys()]
if len(missing_keys) != 0:
    print(missing_keys)
    
missing_keys = [k for k in checkpoint.keys()if k not in model.state_dict()]
if len(missing_keys) != 0:
    print(missing_keys)

model_dict.update({k: v for k, v in checkpoint.items() if k in model.state_dict()})
model.load_state_dict(model_dict) 


print(model)




model.eval()
print("model loaded from file.")

annotation 0
[['54301.6406' '8.42716503']
 ['55424.4492' '58611.2383']
 ['118773.539' '58607']
 ['118781.008' '3.34055328']]




annotation 0
[['50077.6445' '3.68681288']
 ['51752.0508' '81148.6172']
 ['131069.133' '81148.8594']
 ['131069.281' '2.666435']]




annotation 0
[['1.95234442' '2.23125076']
 ['1.68418598' '43517.6797']
 ['49149.6719' '43518.0117']
 ['49150.0273' '1.96491194']]




annotation 0
[['70693.4531' '4.307477']
 ['71314.3594' '51960.9297']
 ['143355.359' '51963.5508']
 ['143356.109' '2.99877715']]




annotation 0
[['6.10493279' '8.13990974']
 ['5.59031487' '61432.9844']
 ['127227.516' '61428.9609']
 ['120928.852' '5.32723618']]




annotation 0
[['69712.4688' '28.2122478']
 ['70098.1953' '31991.7461']
 ['122874.164' '31995.3516']
 ['122874.867' '4.19429016']]




annotation 0
[['46168.3945' '334.207611']
 ['46437.3555' '39673.5039']
 ['122874.188' '39675.4375']
 ['122877.055' '2.60024929']]




annotation 0
[['61620.6523' '459.284363']
 ['61737.1367' '46075.6016']


 computing foreground for patient 36
 computing foreground for patient 37
 computing foreground for patient 38
 computing foreground for patient 39
 computing foreground for patient 40
 computing foreground for patient 41
 computing foreground for patient 42
 computing foreground for patient 43
 computing foreground for patient 44
 computing foreground for patient 45
 computing foreground for patient 46
 computing foreground for patient 47
 computing foreground for patient 48
 computing foreground for patient 49
 computing foreground for patient 50
 computing foreground for patient 51
elapsed time = 2.6578710079193115
['global_DA.Conv1.weight', 'global_DA.Conv2.weight', 'local_DA.Conv1.weight', 'local_DA.Conv2.weight']
PipelineWarwickHER2(
  (module_stg1): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inpla

In [7]:
model.module_stg4[0]

Conv2d(200, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)

# Make the Scheduler In-Turn

In [7]:
tfms = torchvision.transforms.ToTensor()
const_global_info = {
    "mode_trainortest": mode_trainortest,
    "width_bigchunk":2000,
    "heigth_bigchunk":2000,
    "width_smallchunk":224*2,
    "heigth_smallchunk":224*2,
    "num_bigchunkloaders":10,
    "maxlength_queue_smallchunk":100,
    "maxlength_queue_lightdl":10000,
    "interval_resched": 10,
    "core-assignment":{"lightdl":None,
                       "smallchunkloaders":None,
                       "bigchunkloaders":None}
}
#TODO:change the dataset from val_ds to test_ds or whole_dataset
dl_of_statcollector = projutils.datasets.DLWithInTurnSched(
                              dataset=test_ds,\
                              type_bigchunkloader=WSIRandomBigchunkLoader,\
                              type_smallchunkcollector=WSIRandomSmallchunkCollector,\
                              const_global_info=const_global_info,\
                              batch_size=64, tfms=tfms,\
                              flag_grabqueue_onunsched = False
                           ) 
statcollector = BoVWStatCollector(module_pipeline=model, device=device,\
                                lightdl=dl_of_statcollector,\
                                str_collectortype = "accum",\
                                flag_visualizestats=False)
print("statcollector created.")

statcollector created.


In [8]:
statcollector.start_collecting()

 loading initial bigchunks, please wait ....
     bigchunk 0 from 10, please wait ...

     bigchunk 1 from 10, please wait ...

     bigchunk 2 from 10, please wait ...

     bigchunk 3 from 10, please wait ...

     bigchunk 4 from 10, please wait ...

     bigchunk 5 from 10, please wait ...

     bigchunk 6 from 10, please wait ...

     bigchunk 7 from 10, please wait ...

     bigchunk 8 from 10, please wait ...

     bigchunk 9 from 10, please wait ...

The initial loading of bigchunks took 67.31480169296265 seconds.
self.num_calls_to_getflagfinished = 1

 numstats in [0 , 12],     num zeros = 19
self.num_calls_to_getflagfinished = 2

 numstats in [0 , 67],     num zeros = 19
self.num_calls_to_getflagfinished = 3

 numstats in [0 , 77],     num zeros = 19
self.num_calls_to_getflagfinished = 4

 numstats in [0 , 101],     num zeros = 19
 got 10 stats
self.num_calls_to_getflagfinished = 5

 numstats in [0 , 167],     num zeros = 19
self.num_calls_to_getflagfinished = 6

 numstats 

 got 150 stats
self.num_calls_to_getflagfinished = 91

 numstats in [303 , 808],     num zeros = 0
self.num_calls_to_getflagfinished = 92

 numstats in [303 , 808],     num zeros = 0
self.num_calls_to_getflagfinished = 93

 numstats in [303 , 808],     num zeros = 0
self.num_calls_to_getflagfinished = 94

 numstats in [303 , 808],     num zeros = 0
self.num_calls_to_getflagfinished = 95

 numstats in [303 , 808],     num zeros = 0
self.num_calls_to_getflagfinished = 96

 numstats in [303 , 808],     num zeros = 0
 got 160 stats
self.num_calls_to_getflagfinished = 97

 numstats in [303 , 808],     num zeros = 0
self.num_calls_to_getflagfinished = 98

 numstats in [303 , 808],     num zeros = 0
self.num_calls_to_getflagfinished = 99

 numstats in [303 , 808],     num zeros = 0
self.num_calls_to_getflagfinished = 100

 numstats in [303 , 808],     num zeros = 0
self.num_calls_to_getflagfinished = 101

 numstats in [303 , 808],     num zeros = 0
self.num_calls_to_getflagfinished = 102

 nu

self.num_calls_to_getflagfinished = 185

 numstats in [606 , 1000],     num zeros = 0
 got 300 stats
self.num_calls_to_getflagfinished = 186

 numstats in [606 , 1000],     num zeros = 0
self.num_calls_to_getflagfinished = 187

 numstats in [606 , 1000],     num zeros = 0
self.num_calls_to_getflagfinished = 188

 numstats in [606 , 1000],     num zeros = 0
self.num_calls_to_getflagfinished = 189

 numstats in [606 , 1000],     num zeros = 0
self.num_calls_to_getflagfinished = 190

 numstats in [606 , 1000],     num zeros = 0
self.num_calls_to_getflagfinished = 191

 numstats in [606 , 1000],     num zeros = 0
 got 310 stats
self.num_calls_to_getflagfinished = 192

 numstats in [606 , 1000],     num zeros = 0
self.num_calls_to_getflagfinished = 193

 numstats in [606 , 1000],     num zeros = 0
self.num_calls_to_getflagfinished = 194

 numstats in [606 , 1000],     num zeros = 0
self.num_calls_to_getflagfinished = 195

 numstats in [606 , 1000],     num zeros = 0
self.num_calls_to_getfla

self.num_calls_to_getflagfinished = 278

 numstats in [909 , 1000],     num zeros = 0
self.num_calls_to_getflagfinished = 279

 numstats in [909 , 1000],     num zeros = 0
self.num_calls_to_getflagfinished = 280

 numstats in [909 , 1000],     num zeros = 0
self.num_calls_to_getflagfinished = 281

 numstats in [909 , 1000],     num zeros = 0
self.num_calls_to_getflagfinished = 282

 numstats in [909 , 1000],     num zeros = 0
 got 450 stats
self.num_calls_to_getflagfinished = 283

 numstats in [909 , 1000],     num zeros = 0
self.num_calls_to_getflagfinished = 284

 numstats in [909 , 1000],     num zeros = 0
self.num_calls_to_getflagfinished = 285

 numstats in [909 , 1000],     num zeros = 0
self.num_calls_to_getflagfinished = 286

 numstats in [909 , 1000],     num zeros = 0
self.num_calls_to_getflagfinished = 287

 numstats in [968 , 1000],     num zeros = 0
self.num_calls_to_getflagfinished = 288

 numstats in [1000 , 1000],     num zeros = 0


In [9]:
time.sleep(10) #to make sure that the data from subprocesses is transmitted
dict_patient_to_stats = statcollector.get_finalstats()

In [10]:
#compute dict_patient_to_avgencodeddescriptor ====================
dict_patient_to_avgencodeddescriptor = {}
for patient in dict_patient_to_stats.keys():
    if(dict_patient_to_stats[patient] != None):
#         print(dict_patient_to_stats[patient]["sum_encoded_descriptors"].shape)
        avgh_of_patient = dict_patient_to_stats[patient]["sum_encoded_descriptors"]
                #[200 x 14 x 14]
        avgh_of_patient = np.sum(avgh_of_patient, 1) #[200x14]
        avgh_of_patient = np.sum(avgh_of_patient, 1) #[200]
        avgh_of_patient = avgh_of_patient / dict_patient_to_stats[patient]["count"]
        avgh_of_patient = avgh_of_patient/\
                    (dict_patient_to_stats[patient]["sum_encoded_descriptors"].shape[1]*\
                     dict_patient_to_stats[patient]["sum_encoded_descriptors"].shape[2])
        dict_patient_to_avgencodeddescriptor[patient] = avgh_of_patient
    else:
        dict_patient_to_avgencodeddescriptor[patient] = None

#comput dict_patient_to_prob ========
dict_patient_to_prob = {}
for patient in dict_patient_to_stats.keys():
    if(dict_patient_to_stats[patient] != None):
        avgh_of_patient = dict_patient_to_avgencodeddescriptor[patient] #[100]
        with torch.no_grad():
            tensor_hofpatient = torch.from_numpy(avgh_of_patient).float()\
                                .unsqueeze(0).unsqueeze(-1).unsqueeze(-1).to(device)
            prob_of_patient = model.module_stg4(tensor_hofpatient).cpu().numpy()[:,:,0,0]
            dict_patient_to_prob[patient] = prob_of_patient
            

In [11]:
print(tensor_hofpatient.shape)

torch.Size([1, 200, 1, 1])


In [12]:
all_probs, all_her2score, all_predy, all_color = [], [], [], []
for patient in dict_patient_to_prob.keys():
    all_probs.append(dict_patient_to_prob[patient].flatten())
    all_her2score.append(float(patient.dict_records["Her2score"]))
    all_predy.append(np.argmax(dict_patient_to_prob[patient].flatten()))
    print(all_probs[-1])
    print(all_predy[-1])
    print("\n\n")
    
all_probs = np.array(all_probs)

[3.7121372 3.4638326 1.4840536 4.4489536]
3



[ 6.0093193   2.463987    2.1877131  -0.28379184]
0



[4.4741025 3.43693   2.608365  1.0398825]
0



[5.483237   1.3527598  0.71028805 0.3182502 ]
0



[ 6.087124    1.8940307   1.786053   -0.07004127]
0



[2.6350863 3.6259425 1.061762  4.078857 ]
3



[5.5642414e+00 3.4802158e+00 3.1336980e+00 1.3555436e-03]
0



[3.4521782 3.5128973 1.8858912 6.289315 ]
3



[4.9645047  3.043074   1.7570826  0.79824775]
0



[ 6.999161   1.9500606  3.3003154 -1.6813173]
0



[ 6.6932006  1.2646768  3.7402096 -1.008564 ]
0



[ 5.094829    1.7854526   0.92528343 -0.0690111 ]
0



[3.8669586 2.649155  1.5612295 3.081987 ]
0



[3.466089  4.129265  3.673582  3.6940753]
1



[4.767228  3.3812814 2.169173  1.042907 ]
0



[4.447     2.203331  1.8356435 4.951832 ]
3



[2.339197  4.0994735 1.1268044 5.757828 ]
3



[ 6.1371007   2.7152567   2.6666427  -0.35745254]
0



[ 5.3800635   2.6448812   2.3866162  -0.04555795]
0



[2.2878919 3.776606  1.081786  6.17

In [13]:
print([int(u) for u in all_her2score])

[3, 0, 2, 0, 0, 3, 1, 3, 0, 1, 2, 2, 2, 3, 2, 3, 3, 0, 1, 3, 1, 1, 2, 1, 0, 1, 2, 0]


In [14]:
print(all_predy)

[3, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 1, 0, 3, 3, 0, 0, 3, 2, 0, 0, 0, 0, 0, 0, 0]


In [15]:
def confusion_matrix(pred_y, groundtruth_y):
    '''
    Computes confusion matrix.
    Inputs.
        - pred_y: a numpy array or list of containing the predicted labels.
        - groundtruth_y: a numpy array or list containing the groundtruth labels.
    '''
    #manage type variations ====
    if(isinstance(pred_y, list)):
        pass
    elif(isinstance(pred_y, np.ndarray)):
        pred_y = pred_y.flatten().tolist()
    if(isinstance(groundtruth_y, list)):
        pass
    elif(isinstance(groundtruth_y, np.ndarray)):
        groundtruth_y = groundtruth_y.flatten().tolist()
    #get possible labels ====
    list_possiblelabels = list(set(groundtruth_y + pred_y))
    conf_matrix = np.zeros((len(list_possiblelabels), len(list_possiblelabels)))
    assert(len(pred_y) == len(groundtruth_y))
    pred_y = [int(u) for u in pred_y]
    groundtruth_y = [int(u) for u in groundtruth_y]
    for idx_label in range(len(pred_y)):
        conf_matrix[groundtruth_y[idx_label], pred_y[idx_label]] =\
            conf_matrix[groundtruth_y[idx_label], pred_y[idx_label]]+1
    return conf_matrix

def warwickHer2_challengepoints(pred_y, groundtruth_y):
    '''
    The `points` used to evaluate teams in HER2 challenge.
    Inputs.
        - pred_y: a numpy array or list of containing the predicted labels.
        - groundtruth_y: a numpy array or list containing the groundtruth labels.
    '''
    if(isinstance(pred_y, np.ndarray)):
        pred_y = pred_y.flatten().tolist()
    conf_matrix = confusion_matrix(pred_y, groundtruth_y)
    np_scores = np.array([[15 , 15 , 10 , 0],\
                          [15  , 15  , 10 , 0],\
                          [2.5 , 2.5 , 15 , 5],\
                          [0 ,   0   , 10 , 15]])
    toret = np.sum(conf_matrix*np_scores)/len(pred_y)
    return toret

In [16]:
len(train_ds.list_patients)/(len(train_ds.list_patients)+len(test_ds.list_patients))

0.46153846153846156

In [17]:
conf_matrix = confusion_matrix(all_predy, [int(u) for u in all_her2score])
points = warwickHer2_challengepoints(all_predy, [int(u) for u in all_her2score])

In [20]:
conf_matrix

array([[7., 0., 0., 0.],
       [6., 0., 1., 0.],
       [7., 0., 0., 0.],
       [0., 1., 0., 6.]])

In [21]:
#delete the saved foreground maskes ===
list_allpatients = train_ds.list_patients +\
                   val_ds.list_patients +\
                   test_ds.list_patients
for patient in list_allpatients:
    for k in patient.dict_records:
        if(k.startswith("precomputed")):
            patient.dict_records[k] = []
#save the result ======
TODUMP = {
    "dict_patient_to_prob":dict_patient_to_prob,
    "train_ds":train_ds,\
    "val_ds":val_ds,
    "test_ds":test_ds
}
pickle.dump(TODUMP,
            open(
                 "Predictions/predictions_multi_stage__global_trainingtime{}_evalseed{}.pkl"\
                 .format(fname_metainf, val_randomseed_evaluation), "wb"
              )
        )

In [22]:
points*34

379.4642857142857

In [23]:
acc = 100.0*(np.sum(np.diag(conf_matrix))/np.sum(conf_matrix))
print(acc)

46.42857142857143
